In [5]:
from pyopenms import *
import os
import glob

In [ ]:
path= "results/interim"
if not os.path.exists(path):
    os.mkdir(path)

In [24]:
# 1) Precursor Mass Correction

input_original_files = glob.glob('Example_data/*.mzML')

for filename in input_original_files:
    print('Processing: ', filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    delta_mzs= []
    mzs = []
    rts= []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    MzMLFile().store(os.path.join(path, 'PCpeak_' + os.path.basename(filename)), exp)
print('Finished Precursor Mass Correction')

Processing:  Example_data/Actino_1.mzML
Processing:  Example_data/Actino_2.mzML
Finished Precursor Mass Correction


In [25]:
# 2) Feature Detection

input_mzml_files = glob.glob('results/interim/PCpeak_*.mzML')

# 2.1) Mass trace detection

for filename in input_mzml_files:
    print("Mass Trace Detection: ", filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

# 2) Elution peak detection
    print("Elution Peak Detection: ", filename)
    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
     
    if (epd.getParameters().getValue("width_filtering") == "auto"):
          epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
          mass_traces_final = mass_traces_split

# 3) Feature detection
    print("Feature Detection: ", filename)
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
    print(filename[7:-5] + ".featureXML")
    FeatureXMLFile().store(os.path.join(path, os.path.basename(filename)[7:-5] + ".featureXML"), feature_map_FFM)
    
print("Finished Feature Detection")

Mass Trace Detection:  results/interim/PCpeak_Actino_1.mzML
Progress of 'mass trace detection':
-- done [took 0.16 s (CPU), 0.17 s (Wall)] -- 
Elution Peak Detection:  results/interim/PCpeak_Actino_1.mzML
Progress of 'elution peak detection':
Feature Detection:  results/interim/PCpeak_Actino_1.mzML
-- done [took 1.11 s (CPU), 1.11 s (Wall)] -- 
Progress of 'assembling mass traces to features':
/interim/PCpeak_Actino_1.featureXML
Mass Trace Detection:  results/interim/PCpeak_Actino_2.mzML
-- done [took 0.80 s (CPU), 0.77 s (Wall)] -- 
Progress of 'mass trace detection':
Elution Peak Detection:  results/interim/PCpeak_Actino_2.mzML
-- done [took 0.09 s (CPU), 0.11 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 0.61 s (CPU), 0.56 s (Wall)] -- 
Feature Detection:  results/interim/PCpeak_Actino_2.mzML
Progress of 'assembling mass traces to features':/interim/PCpeak_Actino_2.featureXML

-- done [took 0.35 s (CPU), 0.33 s (Wall)] -- 
Finished Feature Detection


In [29]:
# 3) Metabolite Feature Deconvolution

input_feature_files = glob.glob('results/interim/PCpeak_*.featureXML')

for feature_file in input_feature_files:
    feature_map_FFM = FeatureMap()
    FeatureXMLFile().load(feature_file, feature_map_FFM)
    mfd = MetaboliteFeatureDeconvolution()
    mdf_par = mfd.getDefaults()
    mdf_par.setValue("potential_adducts", [b"H:+:0.4",b"Na:+:0.2",b"NH4:+:0.2", b"H-1O-1:+:0.1", b"H-3O-2:+:0.1"])
    mdf_par.setValue("charge_min", 1, "Minimal possible charge")
    mdf_par.setValue("charge_max", 1, "Maximal possible charge")
    mdf_par.setValue("charge_span_max", 1)
    mdf_par.setValue("max_neutrals", 1)
    mfd.setParameters(mdf_par)
        
    feature_map_MFD = FeatureMap()
    cons_map0 = ConsensusMap()
    cons_map1 = ConsensusMap()
    mfd.compute(feature_map_FFM, feature_map_MFD, cons_map0, cons_map1)
    
    ConsensusXMLFile().store(os.path.join(path, 'MFD_' + os.path.basename(feature_file)[7:-11] +".consensusXML"), cons_map0)
    
    FeatureXMLFile().store(os.path.join(path, 'MFD_' + os.path.basename(feature_file)[7:]), feature_map_MFD)

print("Finished Metabolite Feature Deconvolution")

MassExplainer table size: 20
Generating Masses with threshold: -6.90776 ...
done
0 of 198 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 254 to 254
Found 254 putative edges (of 1040) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.028007 seconds,  with objective value: 8.7.
ILP score is: 8.7
Agreeing charges: 340/340
MassExplainer table size: 20
Generating Masses with threshold: -6.90776 ...
done
0 of 30 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 36 to 36
Found 36 putative edges (of 279) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.005534 seconds,  with objective value: 1.68.
ILP score is: 1.68
Agreeing charges: 50/50
Finished Metabolite Feature Deconvolution
